In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [ ]:
import io, sys, math
import numpy as np
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [ ]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [ ]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [ ]:
def softmax(x):
    ### FILL CODE
  max_x = np.max(x)
  exp_x= np.exp(x - max_x)
  sum_exp_x = np.sum(exp_x)
  probs = exp_x/sum_exp_x
  return probs

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [ ]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    for iter in range(niter):
        ### FILL CODE
        total_loss=0.0
        np.random.shuffle(data)
        for (yi,xi) in data:
          s=w@xi
          probs=softmax(s)
          loss=-np.log(probs[yi])
          total_loss+=loss
          error=probs.copy()
          error[yi]-=1.0
          gradient=error.reshape(-1,1)@xi.reshape(-1,1).T
          w=w-0.5*gradient

        avg_loss=total_loss/len(data)
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [ ]:
def predict(w, x):
    ## FILL CODE
    s=w@x
    return np.argmax(s)

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [ ]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    corrects=0
    for (yi,xi) in valid_data:
      pred=predict(w,xi)
      if pred==yi:
        corrects += 1
    return corrects/len(valid_data)

In [ ]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("/content/drive/MyDrive/train1.txt")
train_data = load_data("/content/drive/MyDrive/train1.txt", word_dict, label_dict)
valid_data = load_data("/content/drive/MyDrive/valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **


Validation accuracy: 0.932

